# 날씨와 배추 가격은 어떤 상관관계가 있는 지 예측하는 AI
평균기온(ageTemp), 최저기온(minTemp), 최고기온(maxTemp), 강수량(rainFall)이</br>
배추 가격(agePrice)에 영향을 미칠 경우 가격을 예측

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

In [3]:
# price_data.csv 파일의 데이터를 pandas를 이용해 읽어온다.
price_data = pd.read_csv('price_data.csv')
print(price_data.head()) # 데이터 상위 5개만 추출

       year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0  20100101     -4.9    -11.0      0.9       0.0      2123
1  20100102     -3.1     -5.5      5.5       0.8      2123
2  20100103     -2.9     -6.9      1.4       0.0      2123
3  20100104     -1.8     -5.1      2.2       5.9      2020
4  20100105     -5.2     -8.7     -1.8       0.7      2060


In [4]:
# 읽어온 데이터를 파이썬에서 처리하기 적합하게 numpy 배열로 캐스팅한다.
data = np.array(price_data, dtype=np.float32)
print(data)
print(type(data))

[[ 2.0100100e+07 -4.9000001e+00 -1.1000000e+01  8.9999998e-01
   0.0000000e+00  2.1230000e+03]
 [ 2.0100102e+07 -3.0999999e+00 -5.5000000e+00  5.5000000e+00
   8.0000001e-01  2.1230000e+03]
 [ 2.0100104e+07 -2.9000001e+00 -6.9000001e+00  1.4000000e+00
   0.0000000e+00  2.1230000e+03]
 ...
 [ 2.0171228e+07  2.9000001e+00 -2.0999999e+00  8.0000000e+00
   0.0000000e+00  2.9010000e+03]
 [ 2.0171230e+07  2.9000001e+00 -1.6000000e+00  7.0999999e+00
   6.0000002e-01  2.9010000e+03]
 [ 2.0171232e+07  2.0999999e+00 -2.0000000e+00  5.8000002e+00
   4.0000001e-01  2.9010000e+03]]
<class 'numpy.ndarray'>


In [5]:
# numpy 배열에서 데이터 변화 요인(평균기온, 최저기온, 최대기온, 강수량)으로 사용할 데이터를 뽑아낸다.
xData = data[:,1:5]
print(xData)

[[ -4.9 -11.    0.9   0. ]
 [ -3.1  -5.5   5.5   0.8]
 [ -2.9  -6.9   1.4   0. ]
 ...
 [  2.9  -2.1   8.    0. ]
 [  2.9  -1.6   7.1   0.6]
 [  2.1  -2.    5.8   0.4]]


In [6]:
# numpy 배열에서 결과(평균가격)로 사용할 데이터를 뽑아낸다.
yData = data[:,[5]]
print(yData)

[[2123.]
 [2123.]
 [2123.]
 ...
 [2901.]
 [2901.]
 [2901.]]


In [7]:
# 뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만든다.
# 변화요인(평균기온, 최저기온, 최대기온, 강수량)
X = tf.placeholder(dtype=tf.float32, shape=[None, 4]) 
# 가격(실제값)
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) 

# 다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화 한다.
# 4행 1열 난수를 발생시킨다.
a = tf.Variable(tf.random_uniform([4,1], dtype=tf.float32))
b = tf.Variable(tf.random_uniform([1], dtype=tf.float32))

In [8]:
# 행렬의 곱셈 연산을 이용해 다변인 선형 회귀 모델의 가설 식을 세운다.
y = tf.matmul(X,a) + b

# 오차함수를 만든다.
loss = tf.reduce_mean(tf.square(y-Y))


# 경사하강법 알고리즘을 사용해서 오차 함수 결과를 최소로 하는 식을 만든다.
gradient_descent = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)

In [10]:
# 학습을 시킨 후 학습된 모델을 저장한다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100001):
        loss_, y_, _ = sess.run([loss, y, gradient_descent], feed_dict={X:xData,Y:yData})
#         if i % 5000 == 0:
#             print('Count : %6d, loss = %12.3f, price = %6.1f' %(i,loss_,y_[0]))
            
            

# 학습이 완료되면 학습된 모델을 디스크에 저장한다.
# tf.train.Savor() : 학습된 모델을 디스크로 저장 또는 불러오는 객체를 생성하는 함수


    saver = tf.train.Saver()
    save_path = saver.save(sess, './saved.ckpt')
    print("학습 모델 저장 완료")

학습 모델 저장 완료
